<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-sudoku-solver/blob/main/DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sudoku solver

Authors: Filip Koňařík, Matěj Popďakunik

Abstract: In this project, we wrote a program that can solve Sudoku boards. First, we find the Sudoku in the image, then we use a CNN to recognize the individual digits, and finally, we solve it using a backtracking algorithm.

Methodology: In the final version of the project, we used a custom Convolutional Neural Network (CNN) to recognize individual digits, as it allowed us to achieve the best results.


# DEMO

## Imports and definitions of functions

In [ ]:
import cv2
import numpy as np
import operator
import os
from google.colab.patches import cv2_imshow



import os
import json
import pandas as pd
import albumentations as A
import cv2
from google.colab.patches import cv2_imsho



import cv2
import torch
import numpy as np
import os
import pandas as pd
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchvision import transforms
from google.colab.patches import cv2_imshow
from torch import nn, optim
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import WeightedRandomSampler




!pip install torchmetrics --quiet
!pip install lightning --quiet
import lightning.pytorch as pl
from lightning.pytorch.loggers import CSVLogger
from torchmetrics import ConfusionMatrix, Accuracy




### Sudoku detection

In [ ]:
# Find four corners of sudoku grid using OpenCV functions
def find_corners(sudoku_image):
  sudoku_image = cv2.cvtColor(sudoku_image, cv2.COLOR_BGR2GRAY)
  cv2.GaussianBlur(sudoku_image, (9, 9), 0, sudoku_image)
  cv2.adaptiveThreshold(sudoku_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2, sudoku_image)

  contours = cv2.findContours(sudoku_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contours = contours[0] if len(contours) == 2 else contours[1]
  contours = sorted(contours, key=cv2.contourArea, reverse=True) #sort contours to fing the largest one
  largest_contour = contours[0]

  corners1 = largest_contour[:4].copy()
  corners2 = largest_contour[:4].copy()

  # Find corners of the sudoku
  # We for 2 squares and than check which square has larger area -> works also for rotated images
  for pt in largest_contour:
    if pt[0][0] + pt[0][1] < corners1[0][0][0] + corners1[0][0][1]: corners1[0] = pt
    if pt[0][0] - pt[0][1] > corners1[1][0][0] - corners1[1][0][1]: corners1[1] = pt
    if pt[0][0] + pt[0][1] > corners1[2][0][0] + corners1[2][0][1]: corners1[2] = pt
    if pt[0][0] - pt[0][1] < corners1[3][0][0] - corners1[3][0][1]: corners1[3] = pt
    if pt[0][0] < corners2[0][0][0]: corners2[0] = pt
    if pt[0][1] < corners2[1][0][1]: corners2[1] = pt
    if pt[0][0] > corners2[2][0][0]: corners2[2] = pt
    if pt[0][1] > corners2[3][0][1]: corners2[3] = pt
  if cv2.contourArea(corners1) > cv2.contourArea(corners2):
    return corners1.squeeze()
  else:
    return corners2.squeeze()

#### Example

In [ ]:
#testing
image = cv2.imread("drive/MyDrive/Sudoku/sudoku_data_aug/63.jpg")
corners = find_corners(image)
cv2.drawContours(image, corners, -1, (0,255,0), 15)
cv2_imshow(image)

### Digit classification


In [ ]:
n_classes = 10

#class representing dataset of images of printed sudoku digits from photos
class SudokuNumberDataset(Dataset):
  def __init__(self, root_dir, csv_path, output_size, n_images_to_process=4000):
    self.sudoku_df = pd.read_csv(csv_path) #dataframe containing info about sudoku photos
    self.n_images = n_images_to_process*81 #number of digit images

    #create empty tensors to store digit images and labels in
    self.X = torch.empty((self.n_images, 1, output_size[0], output_size[1]))
    self.y = torch.empty(self.n_images, dtype=torch.long)

    #ids of images from the original dataset
    #images from data augmentation are stored consecutively so we want to sample across the whole dataset
    original_ids = np.linspace(0, 3999, n_images_to_process).astype(int)


    for id in range(n_images_to_process):
      #load greyscale images and corner keypoints
      img_name = os.path.join(root_dir, str(original_ids[id]) + ".jpg")
      image = cv2.cvtColor(cv2.imread(img_name), cv2.COLOR_BGR2GRAY)
      keypoints = np.array(self.sudoku_df.loc[original_ids[id]][4:]).reshape((4, 2)).astype("float32")

      grid_side = max([np.sqrt(p[0]*p[0] + p[1]*p[1]) for p in keypoints]) #find longes distance between corners

      #transform the sudoku image to a square
      keypoints_new = np.array([[0, 0], [grid_side - 1, 0], [grid_side - 1, grid_side - 1], [0, grid_side - 1]], dtype='float32')
      transform = cv2.getPerspectiveTransform(keypoints, keypoints_new)
      image = cv2.warpPerspective(image, transform, (int(grid_side), int(grid_side)))

      #cut the square sudoku image into images of individual cells
      cell_side = grid_side / 9
      for i in range(9):
        for j in range(9):
          top_left = (int(i * cell_side), int(j * cell_side))
          bottom_right = (int((i + 1) * cell_side), int((j + 1) * cell_side))
          number_image = image[top_left[0]:bottom_right[0], top_left[1]:bottom_right[1]]
          number_image = cv2.resize(number_image, output_size)
          self.X[id*81 + i*9 + j] = torch.Tensor(number_image).unsqueeze(0)
          self.y[id*81 + i*9 + j] = torch.tensor(int(self.sudoku_df.loc[original_ids[id]]["nums"][i*9 + j]))

  def __getitem__(self, index):
    return self.X[index], self.y[index]

  def __len__(self):
      return self.n_images

  #normalize the images
  def normalize(self, mean=None, std=None):
    if mean == None and std == None:
      mean = self.X.mean((0, 2, 3))[:, None, None]
      std = self.X.std((0, 2, 3))[:, None, None]
      print("Mean:", mean, "Std:", std)
      self.X = (self.X - mean) / std

In [ ]:
#a convolutional neural network to classify digits from sudoku photos
class DigitClassifier(pl.LightningModule):
  def __init__(self, image_shape, learning_rate=0.1):
    super().__init__()
    self.loss_fn = nn.CrossEntropyLoss()
    self.learning_rate = learning_rate

    #convolutional part of the network
    self.cnn_layers = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
        )

    conv_output_shape = (int(((image_shape[0] - 2) / 2 - 2) / 2),
                         int(((image_shape[1] - 2) / 2 - 2) / 2))
    #linear part of the network
    self.linear_layers = nn.Sequential(
        nn.Linear(in_features=conv_output_shape[0]*conv_output_shape[1]*24, out_features=64),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(in_features=64, out_features=n_classes)
        )

  def forward(self, x):
    y = self.cnn_layers(x)
    y = y.view(y.size(0), -1)
    y = self.linear_layers(y)
    y = F.softmax(y, dim=1)
    return y

  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(x)
    loss = self.loss_fn(y_hat, y)
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(x)
    val_loss = self.loss_fn(y_hat, y)
    accuracy = Accuracy(task="multiclass", num_classes=10).to("cuda:0")
    self.log('val_loss', val_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    self.log('acc', accuracy(y_hat, y), on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return val_loss

  def predict_step(self, batch, batch_idx):
    x, _ = batch
    y_hat = self(x)
    return y_hat

  def configure_optimizers(self):
    return SGD(self.parameters(), lr=self.learning_rate)

After training we get the following confusion matrix

|       | 0     | 1     | 2     | 3     | 4     | 5     | 6     | 7     | 8     | 9     |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| **0** | 10373 | 0     | 0     | 0     | 0     | 1     | 0     | 2     | 0     | 0     |
| **1** | 0     | 612   | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     |
| **2** | 0     | 0     | 683   | 0     | 0     | 0     | 0     | 1     | 0     | 0     |
| **3** | 1     | 0     | 0     | 679   | 0     | 0     | 0     | 0     | 0     | 0     |
| **4** | 0     | 0     | 0     | 0     | 625   | 0     | 0     | 0     | 0     | 0     |
| **5** | 0     | 1     | 0     | 2     | 2     | 629   | 2     | 0     | 0     | 0     |
| **6** | 0     | 0     | 0     | 0     | 0     | 0     | 564   | 0     | 0     | 1     |
| **7** | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 668   | 0     | 0     |
| **8** | 0     | 0     | 0     | 0     | 0     | 0     | 2     | 0     | 738   | 0     |
| **9** | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 614   |


As we can see, the network has a 99% accuracy.


## Showcase